<a href="https://colab.research.google.com/github/goel-vanshika/VehicleClassification/blob/main/MITrial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import division, print_function, absolute_import


In [2]:
!pip install tflearn
!pip install tensorflow
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 6.9 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=2c88416fcf878c4be8695cb7b659b15d4f2853abe282136991782dec1c49f100
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing required libraries

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import tflearn

from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from sklearn.model_selection import train_test_split
from tensorflow.python.framework import ops

Instructions for updating:
non-resource variables are not supported in the long term


Loading the dataset

In [5]:
# Loading test images
from tflearn.data_utils import image_preloader
import numpy as np
# Loading path/class_id image file:
dataset_file = '/content/drive/MyDrive/MI/training/dataset.txt'

Converting the dataset to an array and splitting the dataset into training and testing

In [6]:
image, label = image_preloader(dataset_file, image_shape=(100, 100), mode='file',
                       categorical_labels=True, normalize=True,
                       files_extension=['.jpg', '.png'], filter_channel=True)
image = np.array(image)
label = np.array(label)

X_train,X_test,y_train,y_test = train_test_split(image,label,test_size=0.2)

Image Pre-processing and Image Augmentation

In [7]:
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()


In [8]:
# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

Building the CNN network

In [9]:
# Convolutional network building
network = input_data(shape=[None, 100, 100, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 256, activation='relu')
network = dropout(network, 0.75)
network = fully_connected(network, 256, activation='relu')
network = dropout(network, 0.75)
network = fully_connected(network, 4, activation='softmax')
network = regression(network, optimizer='adam',loss='categorical_crossentropy',learning_rate=0.001)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Building and training the model

In [10]:
# Training using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)


model.fit(X_train, y_train, n_epoch=50, shuffle=True, validation_set=(X_test, y_test), show_metric=True, batch_size=64, run_id='vehicleDetection')
model.save('/content/drive/MyDrive/MI/training/saved_model3/my_model9') 

Training Step: 3599  | total loss: 0.07213 | time: 98.783s
| Adam | epoch: 050 | loss: 0.07213 - acc: 0.9744 -- iter: 4544/4552
Training Step: 3600  | total loss: 0.07547 | time: 105.051s
| Adam | epoch: 050 | loss: 0.07547 - acc: 0.9723 | val_loss: 0.47471 - val_acc: 0.8999 -- iter: 4552/4552
--


Original labels of each image in testing set

In [13]:
y=y_test.tolist()
og=[]
for j in y:
  m=max(j)
  og.append(j.index(m))

print(og)

[1, 3, 1, 1, 1, 1, 2, 0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 3, 1, 1, 2, 3, 3, 1, 1, 1, 2, 1, 0, 0, 1, 1, 0, 2, 1, 0, 0, 1, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 3, 2, 0, 1, 2, 0, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 1, 1, 2, 3, 0, 1, 2, 3, 1, 1, 2, 2, 3, 1, 2, 1, 3, 1, 1, 3, 1, 2, 0, 2, 1, 2, 1, 2, 1, 3, 0, 1, 1, 1, 2, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 3, 1, 1, 2, 0, 1, 1, 3, 2, 1, 3, 2, 0, 0, 1, 3, 0, 2, 1, 2, 3, 2, 2, 1, 3, 1, 1, 2, 1, 3, 1, 1, 1, 3, 2, 1, 1, 1, 0, 3, 1, 3, 2, 2, 3, 1, 0, 2, 1, 1, 1, 0, 0, 2, 2, 2, 3, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 3, 1, 1, 0, 2, 1, 0, 0, 1, 1, 2, 2, 2, 3, 2, 1, 1, 3, 3, 0, 1, 1, 1, 1, 1, 1, 1, 3, 1, 0, 1, 2, 1, 1, 0, 3, 1, 2, 2, 1, 2, 1, 1, 3, 0, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 2, 1, 1, 2, 1, 0, 1, 3, 2, 1, 0, 1, 2, 1, 1, 2, 3, 3, 1, 1, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 3, 2, 1, 2, 1, 0, 3, 2, 1, 2, 1, 1, 1, 1, 1, 1, 0, 3, 2, 2, 0, 1, 2, 0, 2, 1, 1, 2, 2, 0, 1, 3, 2, 1, 1, 2, 2, 1, 3, 1, 1, 2, 0, 3, 1, 2, 1, 1, 0, 3, 1, 1, 2, 2, 1, 2, 3, 

Predicted labels of each image in testing set

In [50]:
pred=model.predict(X_test)
x=pred.tolist()
labels=[]
for i in x:
  m=max(i)
  labels.append(i.index(m))

print(labels)

[1, 1, 1, 2, 1, 1, 2, 0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 0, 1, 1, 2, 1, 1, 2, 2, 1, 2, 0, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 2, 1, 0, 0, 1, 1, 1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 2, 3, 0, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 0, 1, 1, 2, 0, 2, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 1, 2, 0, 2, 2, 1, 2, 1, 1, 2, 0, 0, 1, 1, 0, 2, 1, 2, 1, 2, 2, 1, 1, 1, 0, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 0, 3, 1, 2, 2, 2, 1, 1, 0, 2, 1, 1, 1, 0, 0, 2, 0, 2, 1, 0, 1, 2, 0, 0, 0, 1, 2, 2, 2, 1, 2, 0, 0, 2, 1, 0, 0, 2, 1, 2, 2, 2, 3, 2, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 1, 0, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 2, 1, 2, 2, 1, 0, 2, 1, 2, 1, 0, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 0, 1, 2, 2, 2, 0, 2, 0, 2, 1, 1, 1, 2, 0, 1, 3, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 

0 - Bus
1 - Car
2 - Motorcycle
3 - non-vehicles